# 뉴스 토픽 분류
- 데이터 출처: DACON, 뉴스 토픽 분류 AI 경진대회
    - https://dacon.io/competitions/official/235747/overview/description

In [ ]:
from google.colab import files
up = files.upload()

Saving sample_submission.csv to sample_submission (1).csv
Saving test_data.csv to test_data (1).csv
Saving topic_dict.csv to topic_dict (1).csv
Saving train_data.csv to train_data (1).csv


### 필요한 패키지 임포트

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

### 1. CSV → DataFrame 
- to confirm datas

In [ ]:
train = pd.read_csv('train_data.csv', encoding='utf8')
test = pd.read_csv('test_data.csv', encoding='utf8')
ss = pd.read_csv('sample_submission.csv', encoding='utf8')
topic = pd.read_csv('topic_dict.csv', encoding='utf8')

In [ ]:
train.head(2)

,index,title,topic_idx
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4


In [ ]:
test.head(2)

,index,title
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사


In [ ]:
ss.head(2)

,index,topic_idx
0,45654,0
1,45655,0


### 2. 데이터 전처리


#### < df_train >
- 필요한 컬럼만 가져오기

In [ ]:
train = train[['title','topic_idx']]
train.head(3)

,title,topic_idx
0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4
1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4
2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4


- 중복 및 Null 값 확인하기

In [ ]:
# 중복값 확인
train.shape, train.title.nunique()

((45654, 2), 45654)

In [ ]:
# Null 값 확인
train.isnull().sum().sum()

0

- 한글, 영어, 숫자 이외 데이터 제거

In [ ]:
train.title = train.title.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣|A-Za-z|0-9 ]', '')

In [ ]:
train.title.isnull().sum()

0

- 한자 → 한글로 대체

In [ ]:
!pip install hanja

In [ ]:
import hanja

list(map(lambda x: [hanja.translate(line, 'substitution') for line in train.title], train))[0]
train.head(3)

,title,topic_idx
0,인천핀란드 항공기 결항휴가철 여행객 분통,4
1,실리콘밸리 넘어서겠다구글 15조원 들여 전역 거점화,4
2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4


#### < df_test, df_ss >
- df_test : X_test 값에 해당
- df_ss : y_test 값에 해당

▶ 두 데이터프레임 인덱스 기준으로 합치기

In [ ]:
test = pd.merge(test, ss, how='left')
test.head(2)

,index,title,topic_idx
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영,0
1,45655,어버이날 맑다가 흐려져…남부지방 옅은 황사,0


- 데이터 전처리
    - 위 df_train 과 동일한 과정으로

In [ ]:
# 중복 & Null 값 확인
test.shape, test.title.nunique(), test.isnull().sum().sum()

((9131, 3), 9131, 0)

In [ ]:
# 한글, 영어, 숫자 이외 데이터 제거
test.title = test.title.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣|A-Za-z|0-9 ]', '')
test.title.isnull().sum()

0

In [ ]:
# 한자 → 한글
list(map(lambda x: [hanja.translate(line, 'substitution') for line in test.title], test))[0]
test.head(3)

,index,title,topic_idx
0,45654,유튜브 내달 2일까지 크리에이터 지원 공간 운영,0
1,45655,어버이날 맑다가 흐려져남부지방 옅은 황사,0
2,45656,내년부터 국가RD 평가 때 논문건수는 반영 않는다,0


In [ ]:
# train, test 합치기
data = pd.concat([train, test[['title','topic_idx']]], axis=0, ignore_index=True)
data.tail()

,title,topic_idx
54780,인천 오후 3시35분 대설주의보눈 31cm 쌓여,0
54781,노래방에서 지인 성추행 외교부 사무관 불구속 입건종합,0
54782,40년 전 부마항쟁 부산 시위 사진 2점 최초 공개,0
54783,게시판 아리랑TV 아프리카개발은행 총회 개회식 생중계,0
54784,유영민 과기장관 강소특구는 지역 혁신의 중심지원책 강구,0


### Train/Test dataset 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data.title, data.iloc[:,-1], stratify=data.iloc[:,-1], test_size=0.3, random_state=seed
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((38349,), (16436,), (38349,), (16436,))

### 토큰화

In [ ]:
!pip install --upgrade pip
!pip install konlpy

In [ ]:
from konlpy.tag import Komoran
ko = Komoran()

In [ ]:
stopwords = ['도','는','다','의','가','이','은','한','에','하','고','을','를','인','듯','과','와','네','들','듯','지','임','게']

In [ ]:
X_train = [ko.nouns(t) for t in X_train]
X_train = list(map(lambda s: [item for item in s if item not in stopwords], X_train))

In [ ]:
X_test = [ko.nouns(t) for t in X_test]
X_test = list(map(lambda s: [item for item in s if item not in stopwords], X_test))

### 인코딩 & 패딩

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
t = Tokenizer()
t.fit_on_texts(X_train)

X_train = t.texts_to_sequences(X_train)
X_test = t.texts_to_sequences(X_test)

In [ ]:
# 전체 데이터셋의 길이를 max_len에 맞추기
max_len = max(len(w) for w in X_train)  # 16
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
# 테스트 이외 검증 데이터 만들기
X_test, X_valid, y_test, y_valid = train_test_split(
    X_test, y_test, stratify=y_test, test_size=0.3, random_state=seed
)
X_test.shape, X_valid.shape, y_test.shape, y_valid.shape

((11505, 16), (4931, 16), (11505,), (4931,))

- y_train, y_test, y_valid : one-hot encoding 하기

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train).reshape(-1,7)
y_test = to_categorical(y_test).reshape(-1,7)
y_valid = to_categorical(y_valid)

In [ ]:
y_train.shape, y_test.shape, y_valid.shape

((38349, 7), (11505, 7), (4931, 7))

### 모델 정의/설정/학습

In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### Case 01. LSTM
- embedding dim: 200
- hidden_units: 256

In [ ]:
topic.topic_idx.nunique()

7

In [ ]:
vocab_size = 10000
embedding_dim = 200

model1 = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(256),
    Dense(7, activation='softmax')
])

model1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 16, 200)           2000000   
                                                                 
 lstm_2 (LSTM)               (None, 256)               467968    
                                                                 
 dense_3 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 2,469,767
Trainable params: 2,469,767
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.compile('adam', 'categorical_crossentropy', ['accuracy'])

model_path = 'best-headline-lstm.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=10)

In [ ]:
hist1 = model1.fit(
    X_train, y_train, epochs=30, batch_size=64, 
    validation_data=[X_valid, y_valid],
    callbacks=[mc,es]
)

Epoch 1/30
599/600 [============================>.] - ETA: 0s - loss: 1.0422 - accuracy: 0.6185
Epoch 00001: val_loss improved from inf to 0.81637, saving model to best-headline-lstm.h5
600/600 [==============================] - 14s 15ms/step - loss: 1.0421 - accuracy: 0.6185 - val_loss: 0.8164 - val_accuracy: 0.7021
Epoch 2/30
598/600 [============================>.] - ETA: 0s - loss: 0.6624 - accuracy: 0.7575
Epoch 00002: val_loss did not improve from 0.81637
600/600 [==============================] - 8s 13ms/step - loss: 0.6625 - accuracy: 0.7575 - val_loss: 0.8502 - val_accuracy: 0.6905
Epoch 3/30
598/600 [============================>.] - ETA: 0s - loss: 0.5400 - accuracy: 0.7978
Epoch 00003: val_loss did not improve from 0.81637
600/600 [==============================] - 8s 13ms/step - loss: 0.5402 - accuracy: 0.7978 - val_loss: 0.9551 - val_accuracy: 0.6664
Epoch 4/30
599/600 [============================>.] - ETA: 0s - loss: 0.4514 - accuracy: 0.8283
Epoch 00004: val_loss did n

In [ ]:
best_model = load_model(model_path)
best_model.evaluate(X_test, y_test)

360/360 [==============================] - 4s 8ms/step - loss: 0.8094 - accuracy: 0.7080


[0.8094472289085388, 0.7080399990081787]